In [1]:
# !pip install --quiet datasets
!pip install --quiet scann
!pip install --quiet datasets
!pip install --quiet pipreqsnb
import pandas as pd 
import numpy as np 
# from google.colab import drive
# drive.mount('/content/drive')
import json
import os 
import gzip
# import datasets 
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import *
import tensorflow_hub as hub 

from sklearn.metrics import confusion_matrix

import scann
from data_loader import data_loader
from negative_maker import negative_maker
from model import model

In [3]:
train_df = data_loader('train_eli5').frame_maker()
test_df = data_loader('test_eli5').frame_maker()

In [4]:
# eli5 = datasets.load_dataset('eli5', split = 'train_eli5')

use_hub = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") ## universal sentence encoder model

In [8]:
train_df

,title,selftext,answer,first_answer
0,in football whats the point of wasting the fir...,,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...","Keep the defense honest, get a feel for the pa..."
1,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...","As someone who uses quality Premium, I wish th..."
2,Stars and Visibility,Why do stars in the night's sky seem to disapp...,"{'a_id': ['e1kpw6u'], 'text': ['It's a quirk o...",It's a quirk of the human eye. At the center o...
3,How do we know all the money the government is...,We hear about these large billion dollar bank ...,"{'a_id': ['ceohakd', 'ceoikhs', 'ceoji15', 'ce...",I'm pretty confident most of it isn't going ba...
4,What are good and bad sides of manual and auto...,Please consider I'm not a driver. Automatic se...,"{'a_id': ['clcur3j'], 'text': ['Automatics wei...","Automatics weigh more, so that alone makes gas..."
...,...,...,...,...
272629,Do we all see the same color or my red can be ...,[removed],"{'a_id': ['evmzhep'], 'text': ['Most likely th...",Most likely that we all see colors the same si...
272630,what does that often blue tape that athletes p...,,"{'a_id': ['evmvuxf', 'evmwasy'], 'text': ['It'...",It's called Kinesio tape.\n\nIts supposed to s...
272631,"If cells die of age, where do new ones come from?",Cells tend to die after dividing for a certain...,"{'a_id': ['evmx5tu'], 'text': ['That gets a bi...",That gets a bit complicated because it depends...
272632,why the future of the planet depends on saving...,"I don’t understand, honestly thought they just...","{'a_id': ['evn0fnd', 'evn04ml', 'evn1pat'], 't...",Why are they important? One world: pollination...


In [9]:
train_df = negative_maker(train_df).neg_maker()

In [10]:
# train_df_l = pd.concat([train_neg_1,train_neg_2,train_neg_1])

In [11]:
print(len(train_df))

272634


In [12]:
def distance_calc(y_true, y_pred):
    anchor, positive, negative = tf.split(y_pred, 3, axis=1)
    ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
    an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
    loss = ap_distance - an_distance
    margin = 0
    loss = tf.maximum(loss + margin, 0.0)
    return loss
    # return (ap_distance, an_distance)

In [15]:
print(len(train_df))
print(len(test_df))

272634
24512


In [16]:
triplet_model = model()

In [ ]:

triplet_model.compile(
    optimizer = 'Adam',
    loss = distance_calc
)
y_dummy = np.ones(len(train_df)).reshape(-1,1)
triplet_model.fit([np.array(train_df['title']),
                   np.array(train_df['first_answer']),
                   np.array(train_df['neg_answer'])
                   ],
                   y_dummy,
                   epochs = 4,
                  batch_size = 64*64
                  )


Epoch 1/4
21/67 [========>.....................] - ETA: 7:24 - loss: 4.1789e-04

In [ ]:
use_emb =  triplet_model.get_layer('sentence_encoder')

In [ ]:
q_0 = []
use_emb_test = []
for i in range(len(test_df)):
    # test_quest = test_df['title']
    test_answer = test_df['first_answer'][i]
    y = np.array(use_emb(([test_answer]))).reshape(1,512)

    use_emb_test.append(y)
    # q_0.append(tf.keras.layers.Dot(axes = 1, normalize = True)([test_quest_emb,y]))


In [ ]:
use_emb_test = np.squeeze(np.array(use_emb_test), axis  =1)

In [ ]:
searcher = scann.scann_ops_pybind.builder(use_emb_test, 40, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
test_quest = test_df['title'][0]
test_quest_emb = np.array(use_emb(([test_quest]))).reshape(1,512)



In [ ]:
index, distance = searcher.search(test_quest_emb.ravel())
index

In [ ]:
close_index = []
for i in range(len(test_df)):
    var1 = test_df['title'][i]
    var1 = np.array(use_emb(([var1]))).reshape(1,512)
    index, distance = searcher.search(var1.ravel())
    close_index.append(index)

In [ ]:
# !pipreqsnb . 

In [ ]:
def _compute_precision_recall(targets, predictions, k):

    pred = predictions[:k]
    num_hit = len(set(pred).intersection(set(targets)))
    precision = float(num_hit) / len(pred)
    recall = float(num_hit) / len(targets)
    return precision, recall

N = [1,3,5,10,20]
for t in N:
    precisions = []
    recalls = []
    for i, _k in enumerate(close_index):
        precision, recall = _compute_precision_recall([i], _k,t)
    # print(precision)
        precisions.append(precision)
        recalls.append(recall)

    print('precision @',t, np.mean(precisions))
    print('recalls @',t, np.mean(recalls))
    print(' ')